In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
cd '/content/gdrive/MyDrive/DataScience_project/dataset'

/content/gdrive/MyDrive/DataScience_project/dataset


In [1]:
!pip install pyvi
!pip install underthesea
!pip install demoji
!pip install pyvi
!pip install import_ipynb

  Using cached sklearn_crfsuite-0.3.6-py2.py3-none-any.whl (12 kB)


In [4]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
from gensim.utils import simple_preprocess, to_unicode # thư viện NLP
import import_ipynb
from nlp_preprocessing_lib import *

importing Jupyter notebook from nlp_preprocessing_lib.ipynb


In [5]:
import os
import json
import re
import numpy as np
import pandas as pd
import sqlite3 as sql
import seaborn as sns
import difflib

In [6]:
pd.options.display.max_colwidth = 90

# 1. Product

In [7]:
products = pd.read_csv("ProductRaw.csv")
products.head(2)

,item_id,name,description,rating,price,list_price,brand,group,url,image
0,48102821,"Tai nghe Bluetooth Inpods 12 - Cảm biến vân tay, chống nước,màu sắc đa dạng- 5 màu sắc...",THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời gian pin - Thời gian nghe nhạc liên tục t...,4.0,77000,300000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và Phụ Kiện/Tai Nghe Bluetooth/Tai Nghe Bl...,https://tai-nghe-bluetooth-inpods-12-cam-bien-van-tay-chong-nuoc-mau-sac-da-dang-5-mau...,https://salt.tikicdn.com/cache/280x280/ts/product/9e/af/79/39855aad21aaa6ed4459909c7c0...
1,52333193,Tai nghe bluetooth không dây F9 True wireless Dock Sạc có Led Báo Pin Kép,THÔNG TIN CHI TIẾT\nDung lượng pin 2000mah\nThời gian pin Thời gian nghe nhạc: 100 giờ...,4.5,132000,750000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và Phụ Kiện/Tai Nghe Bluetooth/Tai Nghe Bl...,https://tai-nghe-bluetooth-khong-day-f9-true-wireless-cam-ung-van-tay-dock-sac-co-led-...,https://salt.tikicdn.com/cache/280x280/ts/product/17/c3/53/74638856cc4ec043b46c7614bd0...


In [8]:
# Drop duplicate records
print('Before dropping, records =', products.shape[0])
products.drop_duplicates(inplace=True)
print('After dropping, records =', products.shape[0])

Before dropping, records = 4404
After dropping, records = 4373


In [9]:
products.info()
# Column description has null values.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4373 entries, 0 to 4403
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   item_id      4373 non-null   int64  
 1   name         4373 non-null   object 
 2   description  4370 non-null   object 
 3   rating       4373 non-null   float64
 4   price        4373 non-null   int64  
 5   list_price   4373 non-null   int64  
 6   brand        4373 non-null   object 
 7   group        4373 non-null   object 
 8   url          4373 non-null   object 
 9   image        4373 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 375.8+ KB


In [10]:
# Take a look at rows with null description. Not only description but also name contains information of a product.
# We are going to use both product name & description, so we don't drop these rows with null values
products[products.description.isnull()]

,item_id,name,description,rating,price,list_price,brand,group,url,image
1424,75831435,"Camera Ip EZVIZ C8C hồng ngoại không dây Full HD 2.0 , camera xoay Wifi, có màu ban đê...",NaN,0.0,1465000,3200000,EZVIZ,Máy Ảnh - Máy Quay Phim/Camera Giám Sát/Camera IP,https://camera-ip-ezviz-c8c-hong-ngoai-khong-day-full-hd-2-0-camera-xoay-wifi-co-mau-b...,https://salt.tikicdn.com/cache/280x280/ts/product/e7/1c/27/0541681e3bc53e44456eb20e9a3...
2535,11607917,Màn Hình LED LG 27MK600M-B 27 inch Full HD (1920 x 1080) 5ms 60Hz Radeon FreeSync IPS ...,NaN,0.0,4200000,5500000,LG,Laptop - Máy Vi Tính - Linh kiện/Thiết Bị Văn Phòng - Thiết Bị Ngoại Vi/Màn Hình Máy T...,https://man-hinh-led-lg-27mk600m-b-27-inch-full-hd-1920-x-1080-5ms-60hz-radeon-freesyn...,https://salt.tikicdn.com/cache/280x280/ts/product/d6/61/72/bf2fca6ce6dec3e8377b2a3722d...
3294,16608046,Điện Thoại Di Động Forme A5 - Hãng phân phối chính hãng,NaN,3.7,335000,389000,Forme,Điện Thoại - Máy Tính Bảng/Điện thoại phổ thông,https://dien-thoai-di-dong-forme-a5-hang-phan-phoi-chinh-hang-p16608046.html?src=ss-or...,https://salt.tikicdn.com/cache/280x280/ts/product/51/6c/54/576c31c2a93da609839c354fe32...


In [11]:
# Number of products
print('The number of products:', products.shape[0])

The number of products: 4373


In [12]:
# Number of unique products
print('The number of unique product ID:', products.item_id.count())
# All product ID are unique

The number of unique product ID: 4373


In [13]:
# Number of unique products name
print('The number of unique product Name:', products.name.count())
# All product name are unique

The number of unique product Name: 4373


In [ ]:
# Add product information column

In [14]:
# product_infomation contains all information of a product
products['product_infomation'] = (products.name + products.description + products.brand + products.group).astype(str)

In [15]:
# correct emoji, teencode, wronglist,tokenzie and remove stopwords
products['product_infomation'] = products['product_infomation'].apply(lambda x: clean_text_viet_tokenzie(x, emoji_dict, teen_dict, wrong_lst, stopwords_lst))

In [ ]:
products.product_infomation[0]

'i n p o d s   c ả m _ b i ế n   v â n   c h ố n g   n ư ớ c _ m à u _ s ắ c   đ a _ d ạ n g   m à u _ s ắ c   l ự a   c h ọ n t h ô n g   t i ế t   t h ờ i   t h ờ i   n h ạ c   l i ê n _ t ụ c   t h ờ i   s ạ c   t h ờ i   c h ờ   t h ư ơ n g _ h i ệ u   o e m   x u ấ t _ x ứ   t h ư ơ n g _ h i ệ u   q u ố c   đ ộ   n h ạ y _ c ả m _ b i ế n   v â n   c ắ m   t r ọ n g _ l ư ợ n g   t h ờ i   t h ờ i   n h ạ c   l i ê n _ t ụ c   s k u   m ô _ t ả   s ả n _ p h ẩ m   i n p o d   p h i ê n _ b ả n   n â n g _ c ấ p   t h i ế t _ k ế   t ỉ _ l ệ   c h u ẩ n   a i r p o d   h ã n g   l ư ợ c   n ú t   b ấ m   t h â n   n ú t   c ả m _ ứ n g   d ễ _ d à n g   t h u ậ n _ t i ệ n   t á c   c u ộ c _ g ọ i   n h ạ c   d ễ _ d à n g   c h ạ m   k ế t _ n ố i   v ô _ c ù n g   ổ n _ đ ị n h   k ế t _ n ố i   v ô _ c ù n g   ổ n _ đ ị n h   k ế t _ n ố i   d o c k   s ạ c _ c h ấ t   â m   t h ờ i _ l ư ợ n g   c ả i _ t h i ệ n   t ố i _ ư u   d o c k   s ạ c   t i ệ n _ l ợ i   s ạ c   t h

In [16]:
# Save products
products.to_csv('Product_clean.csv', index=False)

# 2. Review

In [17]:
reviews = pd.read_csv("ReviewRaw.csv")
reviews.head(2)

,customer_id,product_id,name,full_name,created_time,rating,title,content
0,709310,10001012,Lân Nguyễn Hoàng,Lân Nguyễn Hoàng,NaN,3,Ko dùng đc thẻ nhớ,"Lúcđầu quên thông tin nên dùng 512gb thì ko đc, đọc kĩ lại là 128 thì khởi tạo cũng ko..."
1,10701688,10001012,Nguyễn Khánh Hòa,Nguyễn Khánh Hòa,NaN,5,Cực kì hài lòng,"Tiki giao hàng nhanh. Sản phẩm đúng như mô tả, giá cả khá tốt. Camera bắt chuyển động ..."


In [18]:
# Drop duplicate records
print('Trước khi dropping, review =', reviews.shape[0])
reviews.drop_duplicates(inplace=True)
print('Sau khi dropping, review =', reviews.shape[0])

Trước khi dropping, review = 364099
Sau khi dropping, review = 361750


In [19]:
reviews.info()
# Column ... have null values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 361750 entries, 0 to 364098
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   customer_id   361750 non-null  int64 
 1   product_id    361750 non-null  int64 
 2   name          361322 non-null  object
 3   full_name     327147 non-null  object
 4   created_time  116225 non-null  object
 5   rating        361750 non-null  int64 
 6   title         361721 non-null  object
 7   content       165686 non-null  object
dtypes: int64(3), object(5)
memory usage: 24.8+ MB


In [20]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns


In [21]:
missing_values_table(reviews)

Your selected dataframe has 8 columns.
There are 5 columns that have missing values.


,Missing Values,% of Total Values
created_time,245525,67.9
content,196064,54.2
full_name,34603,9.6
name,428,0.1
title,29,0.0


In [22]:
# Drop feature created_time and full_name
reviews.drop(['full_name','created_time'],axis=1, inplace=True)

In [23]:
# create review_info which contains essential keywords from title and content
review_information = (reviews.title + " " + reviews.content).astype('str')
# correct emoji, teencode, wronglist,tokenzie and remove stopwords
reviews['review_information'] = review_information.apply(lambda x: clean_text_viet_tokenzie(x, emoji_dict, teen_dict, wrong_lst, stopwords_lst))

In [25]:
# Save reviews
reviews.to_csv('Review_new.csv', index=False)